
# Sistema de Recomendación Avanzada con Python

Los sistemas de recomendación generalmente se basan en conjuntos de datos más grandes y específicamente necesitan organizarse de una manera particular. Tendremos un proceso de tutorización más intensivo sobre la creación de un sistema de recomendación con Python con el mismo conjunto de datos sobre películas.

*Nota: Las matemáticas reales detrás de los sistemas de recomendación son bastante profundas en lo que respecta al Algebra Lineal.*
___

## Métodos utilizados

Los dos tipos más comunes de sistemas de recomendación son ** Basado en contenido ** y ** Filtrado colaborativo (CF) **.

* El filtrado colaborativo produce recomendaciones basadas en el conocimiento de la actitud de los usuarios hacia los ítems, es decir, utiliza la "sabiduría de la multitud" para recomendar ítems.
* Los sistemas de recomendación basados ​​en el contenido se centran en los atributos de los artículos y le dan recomendaciones basadas en la similitud entre ellos.

## Filtrado colaborativo

En general, el filtrado colaborativo (CF) se usa con más frecuencia que los sistemas basados ​​en el contenido porque generalmente ofrece mejores resultados y es relativamente fácil de entender (desde una perspectiva general de implementación). El algoritmo tiene la capacidad de hacer un aprendizaje de características por sí mismo, lo que significa que puede comenzar a aprender por sí mismo qué características usar.

CF se puede dividir en ** Filtrado colaborativo basado en memoria ** y ** Filtrado colaborativo basado en modelo **.

En este tutorial, implementaremos CF basado en modelo mediante el uso de la descomposición de valores singulares (SVD) y la CF basado en memoria mediante el cálculo de la similitud del coseno.

## Los datos

Utilizaremos el famoso conjunto de datos MovieLens, que es uno de los conjuntos de datos más comunes utilizados al implementar y probar motores de recomendaciones. Contiene 100k clasificaciones de películas de 943 usuarios y una selección de 1682 películas.

Puede descargar el conjunto de datos [aquí](http://files.grouplens.org/datasets/movielens/ml-100k.zip) o simplemente usar el archivo u.data que ya está incluido en esta carpeta.

____
## Comenzando

Vamos a importar algunas bibliotecas que necesitaremos:


In [2]:
import numpy as np
import pandas as pd

Podemos leer en el archivo ** u.data **, que contiene el conjunto de datos completo. Puede leer una breve descripción del conjunto de datos [aquí](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

Tenga en cuenta cómo especificamos el argumento separador para un archivo separado por tabuladores.

In [3]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

Echemos un vistazo rápido a los datos.

In [4]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


Tenga en cuenta que solo tenemos el item_id, no el nombre de la película. Podemos usar el archivo csv Movie_ID_Titles para tomar los nombres de las películas y fusionarlo con este DataFrame:

In [5]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


A continuación, combine los DataFrames:

In [6]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


Ahora echemos un vistazo rápido a la cantidad de usuarios y películas.

In [7]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Usuarios: '+ str(n_users))
print('Num. of Películas: '+str(n_items))

Num. of Usuarios: 944
Num. of Películas: 1682


## División en conjunto de prueba y de entrenamiento

Los sistemas de recomendación por su propia naturaleza son muy difíciles de evaluar, pero aún así le mostraremos cómo evaluarlos. Para hacer esto, dividiremos nuestros datos en dos conjuntos. Sin embargo, no haremos nuestra clásica división en X_train, X_test, y_train, y_test. En cambio, podemos simplemente dividir los datos en dos conjuntos de datos:

In [8]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

C:\Users\Juan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Filtrado colaborativo basado en memoria

Los enfoques de filtrado colaborativo basado en la memoria se pueden dividir en dos secciones principales: ** filtrado de ítems de usuario ** y ** filtrado de ítems de ítems **.

Un * filtrado de ítems de usuario * tomará un usuario en particular, buscará usuarios que sean similares a ese usuario en función de la similitud de calificaciones y recomendará los artículos que les gustaron a esos usuarios similares.

Por el contrario, el * filtrado de ítems de ítems * tomará un ítem, buscará usuarios a los que les haya gustado y buscará otros ítems que también les hayan gustado a esos usuarios o usuarios similares. Toma ítems y genera otros ítems como recomendaciones.

* * Filtrado colaborativo de ítems de artículos *: "A los usuarios que les gustó este ítem también les gustó ..."
* * Filtrado colaborativo de ítems de usuario *: "A los usuarios que son similares a ti también les gustó ..."

En ambos casos, se crea una matriz de ítems-usuario que se creó a partir de todo el conjunto de datos.

Dado que hemos dividido los datos en pruebas y entrenamiento, tendremos que crear dos matrices `` [943 x 1682] `` (todos los usuarios por todas las películas).

La matriz de entrenamiento contiene el 75% de las calificaciones y la matriz de prueba contiene el 25% de las calificaciones.

Después de que haya creado la matriz de usuario-ítem, calcule la similitud y cree una matriz de similitud.

Los valores de similitud entre elementos en el * Filtrado Colaborativo ítem-ítem * se miden al observar a todos los usuarios que han calificado ambos elementos.

Para el * Filtrado colaborativo de usuario-ítem *, los valores de similitud entre los usuarios se miden al observar todos los ítems que califican ambos usuarios.

Una métrica de distancia comúnmente utilizada en los sistemas de recomendación es *similitud de coseno*, donde las clasificaciones se ven como vectores en el espacio ``n`` dimensional y la similitud se calcula en función del ángulo entre estos vectores.
La similitud del coseno para los usuarios *a* y *m* se puede calcular usando la siguiente fórmula, donde se toma el producto punto del vector de usuario *$u_k$ * y el vector de usuario *$u_a$* y se divide multiplicando la longitud Euclidiana de los vectores.
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

Para calcular la similitud entre ítems *m* y *b* se usa la fórmula:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Su primer paso será crear la matriz usuario-ítem. Como tiene datos de prueba y entrenamiento, necesita crear dos matrices.

In [17]:
#Crear dos matrices usuario-ítem, uno para entrenamiento y otro para prueba
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

Puede usar la función [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) de sklearn para calcular la similitud del coseno. Tenga en cuenta que la salida variará de 0 a 1, ya que las calificaciones son todas positivas.

In [18]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

El siguiente paso es hacer predicciones. Ya ha creado matrices de similitud: `user_similarity` y `item_similarity` y, por lo tanto, puede hacer una predicción aplicando la siguiente fórmula para CF basada en el usuario:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

Puede ver la similitud entre los usuarios *k* y *a* como ponderaciones que se multiplican por las clasificaciones de un usuario similar *a* (corregido para la calificación promedio de ese usuario). Necesitará normalizarlo para que las calificaciones se mantengan entre 1 y 5 y, como último paso, sume las calificaciones promedio para el usuario que está tratando de predecir.

La idea aquí es que algunos usuarios tienden a dar calificaciones altas o bajas a todas las películas. La diferencia relativa en las calificaciones que dan estos usuarios es más importante que los valores absolutos. Para dar un ejemplo: supongamos que el usuario *k* le otorga 4 estrellas a sus películas favoritas y 3 estrellas a todas las demás buenas películas. Supongamos ahora que otro usuario *t* califica las películas que le gustan con 5 estrellas, y las películas con las que se durmió con 3 estrellas. Estos dos usuarios podrían tener un gusto muy similar pero tratar el sistema de clasificación de manera diferente.

Al hacer una predicción para CF basada en elementos, no es necesario corregir la calificación promedio de los usuarios, ya que el usuario de la consulta se usa para hacer predicciones.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [19]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [20]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

### Evaluación
Hay muchas métricas de evaluación, pero una de las métricas más utilizadas para evaluar la precisión de las calificaciones pronosticadas es *Root Mean Squared Error (RMSE)*.
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

Puede usar la función [mean_square_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE) de `sklearn`, donde el RMSE es simplemente la raíz  cuadrada de MSE. Para leer más sobre diferentes métricas de evaluación, puede echar un vistazo a [este articulo](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf). 

Como solo quiere considerar las calificaciones pronosticadas que están en el conjunto de datos de prueba, filtrará todos los demás elementos en la matriz de predicción con `prediction [ground_truth.nonzero ()]`.

In [21]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [22]:
print('CF basado en usuario RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('CF basado en ítem RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

CF basado en usuario RMSE: 3.1232607587671155
CF basado en ítem RMSE: 3.4519428133183143


Los algoritmos basados en memoria son fáciles de implementar y producen una calidad de predicción razonable. El inconveniente de la CF basada en la memoria es que no se adapta a los escenarios del mundo real y no aborda el conocido problema de arranque en frío, es decir, cuando un nuevo usuario o un nuevo elemento ingresa al sistema. Los métodos de CF basados en modelos son escalables y pueden lidiar con un nivel de dispersión más alto que los modelos basados en memoria, pero también sufren cuando nuevos usuarios o elementos que no tienen ninguna calificación ingresan al sistema. Me gustaría agradecer a Ethan Rosenthal por su [publicación](http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/) sobre el filtrado colaborativo basado en la memoria.

# Filtrado colaborativo basado en modelos

El filtrado colaborativo basado en modelos se basa en la **factorización de matriz (MF)** que ha recibido una mayor exposición, principalmente como un método de aprendizaje no supervisado para la descomposición de variables latentes y la reducción de la dimensionalidad. La factorización matricial se usa ampliamente para los sistemas de recomendación, en los que puede lidiar mejor con la escalabilidad y la escasez que con los CF basados ​​en memoria. El objetivo de MF es conocer las preferencias latentes de los usuarios y los atributos latentes de los elementos de las calificaciones conocidas (funciones de aprendizaje que describen las características de las calificaciones) para luego predecir las calificaciones desconocidas a través del producto puntual de las características latentes de los usuarios y los elementos. Cuando tiene una matriz muy dispersa, con muchas dimensiones, al realizar la factorización de la matriz puede reestructurar la matriz de elementos de usuario en una estructura de bajo rango, y puede representar la matriz mediante la multiplicación de dos matrices de bajo rango, donde las filas contienen el vector latente. Usted ajusta esta matriz para aproximar su matriz original, lo más cerca posible, multiplicando las matrices de bajo rango juntas, lo que llena las entradas que faltan en la matriz original.

Calculemos el nivel de dispersión del conjunto de datos MovieLens:

In [23]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('El nivel de dispersión de MovieLens100K es ' +  str(sparsity*100) + '%')

El nivel de dispersión de MovieLens100K es 93.7%


Para dar un ejemplo de las preferencias latentes aprendidas de los usuarios y elementos: digamos que para el conjunto de datos MovieLens tiene la siguiente información: _(user id, age, location, gender, movie id, director, actor, language, year, rating)_. Al aplicar la factorización matricial, el modelo aprende que las características importantes del usuario son _age group (under 10, 10-18, 18-30, 30-90)_, _location_ y _gender_, y para películas, aprende que _decade_, _director_ y _actor_ son los más importantes. Ahora, si observa la información que ha almacenado, no existe ninguna característica como _decade_, pero el modelo puede aprender por su cuenta. El aspecto importante es que el modelo CF solo utiliza datos (user_id, movie_id, rating) para aprender las características latentes. Si hay poca información disponible, el modelo de CF basado en el modelo predecirá mal, ya que será más difícil aprender las características latentes. 

Los modelos que utilizan tanto las clasificaciones como las funciones de contenido se denominan sistemas de recomendación híbridos en los que se combinan tanto el filtrado colaborativo como los modelos basados en contenido. Los **sistemas de recomendación híbridos** por lo general muestran una mayor precisión que los modelos de filtrado colaborativo o los modelos basados en contenido por sí solos: son capaces de abordar mejor el problema del arranque en frío, ya que si no tiene ninguna clasificación para un usuario o un elemento, podría usar los metadatos del usuario o elemento para hacer una predicción.

### SVD
Un método de factorización de matriz bien conocido es la **Descomposición de valores singulares (SVD)**. El filtrado colaborativo se puede formular aproximando una matriz `X` mediante la descomposición de valores singulares. El equipo ganador en el concurso del Premio Netflix utilizó modelos de factorización de matriz SVD para producir recomendaciones de productos, para más información recomiendo leer artículos: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) y [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).
La ecuación general puede ser expresada como:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Dada la matriz de tamaño `m x n`, `X`:
* *`U`* es una *`(m x r)`* matriz ortogonal
* *`S`* es una *`(r x r)`* matriz diagonal con números reales no negativos en la diagonal
* *V^T* es una *`(r x n)`* matriz ortogonal

Los elementos en la diagonal de `S` son conocidos como *valores singulares de `X`*. 


La matriz *`X`* se puede factorizar a *`U`*, *`S`* y *`V`*. La matriz *`U`* representa los vectores de características correspondientes a los usuarios en el espacio de características ocultas y la matriz *`V`* representa los vectores de características correspondientes a los elementos en el espacio de características ocultas.

Ahora puede hacer una predicción tomando el producto punto de *`U`*, *`S`* y *`V^T`*.


In [25]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('CF basado en el usuario, MSE: ' + str(rmse(X_pred, test_data_matrix)))

CF basado en el usuario, MSE: 2.7161014785904958


Carelessly addressing only the relatively few known entries is highly prone to overfitting. SVD can be very slow and computationally expensive. More recent work minimizes the squared error by applying alternating least square or stochastic gradient descent and uses regularization terms to prevent overfitting. Alternating least square and stochastic gradient descent methods for CF will be covered in the next tutorials.
Tratar de manera descuidada solo las relativamente pocas entradas conocidas es altamente propenso al sobreajuste. La SVD puede ser muy lenta y computacionalmente costosa. El trabajo más reciente minimiza el error cuadrado al aplicar un descenso de gradiente estocástico o cuadrado mínimo alternativo y utiliza términos de regularización para evitar el sobreajuste. 

Revisión:

* Hemos cubierto cómo implementar métodos simples de **Filtrado colaborativo**, tanto CF basados en memoria como CF basados en modelos.
* **Los modelos basados en memoria** se basan en la similitud entre elementos o usuarios, donde utilizamos la similitud de coseno.
* **Los CF basados en modelos** se basan en la factorización de la matriz, donde utilizamos la SVD para factorizar la matriz.
* Crear sistemas de recomendación que tengan un buen desempeño en situaciones de arranque en frío (donde hay poca información disponible sobre nuevos usuarios y artículos) sigue siendo un desafío. El método de filtrado colaborativo estándar funciona mal en tales configuraciones 

## ¿Buscando más?

Si desea abordar su propio análisis de sistema de recomendaciones, consulte estos conjuntos de datos. Nota: los archivos son bastante grandes en la mayoría de los casos, no todos los enlaces pueden permanecer al día para alojar los datos, pero la mayoría de ellos aún funcionan. O simplemente Googlee para su propio conjunto de datos.

**Recomendación de películas:**

MovieLens - Movie Recommendation Data Sets http://www.grouplens.org/node/73

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Jester - Movie Ratings Data Sets (Collaborative Filtering Dataset) http://www.ieor.berkeley.edu/~goldberg/jester-data/

Cornell University - Movie-review data for use in sentiment-analysis experiments http://www.cs.cornell.edu/people/pabo/movie-review-data/

**Recomendación de música:**

Last.fm - Music Recommendation Data Sets http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Audioscrobbler - Music Recommendation Data Sets http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

Amazon - Audio CD recommendations http://131.193.40.52/data/

**Recomendación de libros:**

Institut für Informatik, Universität Freiburg - Book Ratings Data Sets http://www.informatik.uni-freiburg.de/~cziegler/BX/
Food Recommendation:

Chicago Entree - Food Ratings Data Sets http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data
Merchandise Recommendation:

**Recomendación de salud:**

Nursing Home - Provider Ratings Data Set http://data.medicare.gov/dataset/Nursing-Home-Compare-Provider-Ratings/mufm-vy8d

Hospital Ratings - Survey of Patients Hospital Experiences http://data.medicare.gov/dataset/Survey-of-Patients-Hospital-Experiences-HCAHPS-/rj76-22dk

**Recomendación de citas:**

www.libimseti.cz - Dating website recommendation (collaborative filtering) http://www.occamslab.com/petricek/data/
Scholarly Paper Recommendation:

National University of Singapore - Scholarly Paper Recommendation http://www.comp.nus.edu.sg/~sugiyama/SchPaperRecData.html
